In [1]:
! pip install jupyter-dash

In [2]:
# Import required libraries
import pandas as pd
from jupyter_dash import JupyterDash
from dash import html
from dash import dcc
from dash.dependencies import Input, Output
import plotly.express as px

In [3]:
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

In [4]:
spacex_df.head()

,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


In [5]:
spacex_df[spacex_df['Launch Site']== 'CCAFS LC-40']

,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0
5,5,7,CCAFS LC-40,0,3170.0,F9 v1.1,v1.1
6,6,8,CCAFS LC-40,0,3325.0,F9 v1.1,v1.1
7,7,9,CCAFS LC-40,0,2296.0,F9 v1.1,v1.1
8,8,10,CCAFS LC-40,0,1316.0,F9 v1.1,v1.1
9,9,11,CCAFS LC-40,0,4535.0,F9 v1.1,v1.1


In [6]:
# Create a dash application
app = JupyterDash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                html.Div([
                                           
                                            html.H2('Launch Site:', style={'margin-right': '2em'}),
                                            dcc.Dropdown(id='site-dropdown',
                                            options=[{'label': 'All Sites', 'value': 'ALL'},
                                                     {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                                     {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
                                                     {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                                     {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                                     ],
                                            value='ALL',placeholder='Select a Launch Site here',
                                            searchable=True) 
                                            ]),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),
                                
                                html.Div([
                                html.H2('Payload Slider:', style={'margin-right': '2em'}),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',min=0,max=10000,step=1000
                                ,marks={0: '0', 100: '100'},
                                value=[min_payload, max_payload])
                                ]),
                                html.Br(),
                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback([Output(component_id='success-pie-chart', component_property='figure'),
              Output(component_id='success-payload-scatter-chart', component_property='figure')],
              [Input(component_id='site-dropdown', component_property='value'),
               Input(component_id='payload-slider', component_property='value')])

def get_pie_chart(entered_site,slider):
    low , high = slider
    if entered_site == 'ALL':
        fig_1 = px.pie(spacex_df, values='class', names='Launch Site', title='Pie Chart')
        fig_2 = px.scatter(spacex_df[(spacex_df['Payload Mass (kg)']<high) & (spacex_df['Payload Mass (kg)']>low)], x = 'Payload Mass (kg)' ,y='class', title='Scatter Plot')
        return fig_1,fig_2
    else:
        filtered_df = spacex_df[spacex_df['Launch Site']==entered_site]
        lenght = filtered_df[(filtered_df['Payload Mass (kg)']<high) & (filtered_df['Payload Mass (kg)']>low)].shape[0]
        values = [1 for i in range(lenght)]
        fig_1 = px.pie(filtered_df[(filtered_df['Payload Mass (kg)']<high) & (filtered_df['Payload Mass (kg)']>low)], values=values, names='class', title='Pie Chart')
        fig_2 = px.scatter(filtered_df[(filtered_df['Payload Mass (kg)']<high) & (filtered_df['Payload Mass (kg)']>low)], x = 'Payload Mass (kg)', y='class',title='Scatter Plot') 
        return fig_1,fig_2

In [7]:
app.run_server(mode='external')

Dash app running on http://127.0.0.1:8050/
